In [1]:
#@title Install and Import Dependencies
import torch
torch.set_num_threads(1)

from IPython.display import Audio
from pprint import pprint
# download example
torch.hub.download_url_to_file('https://models.silero.ai/vad_models/en.wav', 'en_example.wav')

100.0%


In [2]:
USE_ONNX = False # change this to True if you want to test onnx model
if USE_ONNX:
    !pip install -q onnxruntime

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True,
                              onnx=USE_ONNX)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /hpc_stor03/sjtu_home/ziyang.ma/.cache/torch/hub/master.zip


In [3]:
get_speech_timestamps

<function utils_vad.get_speech_timestamps(audio: torch.Tensor, model, threshold: float = 0.5, sampling_rate: int = 16000, min_speech_duration_ms: int = 250, max_speech_duration_s: float = inf, min_silence_duration_ms: int = 100, window_size_samples: int = 512, speech_pad_ms: int = 30, return_seconds: bool = False, visualize_probs: bool = False, progress_tracking_callback: Callable[[float], NoneType] = None)>

In [4]:
save_audio

<function utils_vad.save_audio(path: str, tensor: torch.Tensor, sampling_rate: int = 16000)>

In [6]:
collect_chunks

<function utils_vad.collect_chunks(tss: List[dict], wav: torch.Tensor)>

In [5]:
SAMPLING_RATE = 16000
wav = read_audio('/hpc_stor03/sjtu_home/ziyang.ma/data/demo_pandas/pandas.wav', sampling_rate=SAMPLING_RATE)
# get speech timestamps from full audio file
speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=SAMPLING_RATE)
pprint(speech_timestamps)

/hpc_stor03/sjtu_home/ziyang.ma/miniconda3/envs/slam/lib/python3.9/site-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %669 : int[] = prim::profile_ivalue(%667)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  return forward_call(*args, **kwargs)


[{'end': 263136, 'start': 1568},
 {'end': 417248, 'start': 282144},
 {'end': 480224, 'start': 455712},
 {'end': 550880, 'start': 494112},
 {'end': 711648, 'start': 564768},
 {'end': 736736, 'start': 715296},
 {'end': 775648, 'start': 770592},
 {'end': 799712, 'start': 783392},
 {'end': 823776, 'start': 810528},
 {'end': 1128416, 'start': 850976},
 {'end': 1214944, 'start': 1196064},
 {'end': 1256416, 'start': 1236512},
 {'end': 1487328, 'start': 1463328},
 {'end': 1524192, 'start': 1501216},
 {'end': 1546720, 'start': 1533472},
 {'end': 1566688, 'start': 1560608},
 {'end': 1591264, 'start': 1569824},
 {'end': 1625056, 'start': 1615392},
 {'end': 1733600, 'start': 1722400},
 {'end': 1794016, 'start': 1750560},
 {'end': 1918944, 'start': 1808928},
 {'end': 2020320, 'start': 2015264},
 {'end': 2237920, 'start': 2164768},
 {'end': 2274784, 'start': 2244128},
 {'end': 2287584, 'start': 2279456},
 {'end': 2554848, 'start': 2313760},
 {'end': 2576864, 'start': 2559520},
 {'end': 2637792, 'sta

In [10]:
import os
for timestamp in speech_timestamps:
    save_audio(
        os.path.join('/hpc_stor03/sjtu_home/ziyang.ma/data/demo_pandas/vad', str(timestamp['start'])+'_'+str(timestamp['end'])+'.wav'),
        collect_chunks([timestamp], wav),
        sampling_rate=16000
    )

In [1]:
import whisper

model = whisper.load_model("/hpc_stor03/sjtu_home/zheshu.song/.cache/whisper/large-v3.pt")
result = model.transcribe("/hpc_stor03/sjtu_home/ziyang.ma/data/demo_pandas/pandas.wav")
print(result["text"])

 Wings of surveillance! Why do you do that? Do what? Just because you say wings of before something, it doesn't mean that you're doing a special move. It's like me saying, antenna of power, or thorax of making sandwiches. Wings of disagreement. Whoa! There! Master Bear, Master Chicken, Master Crog, what are you doing out here? Look out! Chained creatures attacked our villages! We tracked them here! Stop! They must be in there. Master Shifu strongly advised us... What? Not to engage. Gotta get in there. But Master Shifu said... You're seriously afraid? Even Master Chicken's going in there! And he's a chicken. That's it. I'm going in. Mantis, you're in trouble. I'm not. I'm not. I'm not. I'm not. Mantis, we have orders not to... They need our help. Come on. I'll go high, you go low. No, wait. Fear the bug! All right, you little... Hold on, buddy. I'm coming! Antenna of power! Ah! It didn't work! Mantis! Mantis? Mantis. Your chi is strong. Just like your friend, the bug. Mantis! Don't wor